In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk

Loading the training dataset.

In [2]:
df=pd.read_csv("/content/train.csv")

In [3]:
df.head()

,Category,Text
0,Accountant,education omba executive leadership university...
1,Accountant,howard gerrard accountant deyjobcom birmingham...
2,Accountant,kevin frank senior accountant inforesumekraftc...
3,Accountant,place birth nationality olivia ogilvy accounta...
4,Accountant,stephen greet cpa senior accountant 9 year exp...


In [4]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [5]:
!pip install contractions
!pip install num2words

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.5/163.5 kB 11.8 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=915d2da4882598a4083a5c5e71c1ceb4d6118eebe2c1d08c6175754c8f26a7f1
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
Successfully built docopt


In [6]:
import re
import contractions
from num2words import num2words
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()

    # Expand contractions (e.g., "don't" → "do not")
    text = contractions.fix(text)

    # Convert numbers to words (e.g., "100" → "one hundred")
    def num_to_words(match):
        num_str = match.group().replace(',', '')  # Remove commas
        try:
            if '.' in num_str:
                return num2words(float(num_str))
            else:
                return num2words(int(num_str))
        except:
            return num_str

    text = re.sub(r'\d+(?:,\d+)*(?:\.\d+)?', num_to_words, text)

    # Remove non-alphabetical characters (keeping spaces)
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [w for w in words if w not in stop_words]

    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(w) for w in words]  # e.g., "better" → "good"

    return ' '.join(words)

### Preprocessing Steps Performed by `preprocess_text(text)`

1. **Lowercasing**  
   Converts all characters in the text to lowercase to ensure uniformity.  
2. **Removing Special Characters and Numbers**
   Uses regex to strip out non-alphabetical characters (excluding spaces).
3. **Stop-word Removal** Removes common English stopwords that often carry little semantic meaning.

4. **Lemmatization** Reduces words to their base form using WordNetLemmatizer.

5. **Normalisation** contraction expansion, number replacement

Skipping **Augumentation** for now, if we get a low performance will come back to this step.

In [7]:
df['Text'] = df['Text'].apply(preprocess_text)

In [8]:
df.head()

,Category,Text
0,Accountant,education omba executive leadership university...
1,Accountant,howard gerrard accountant deyjobcom birmingham...
2,Accountant,kevin frank senior accountant inforesumekraftc...
3,Accountant,place birth nationality olivia ogilvy accounta...
4,Accountant,stephen greet cpa senior accountant nine year ...


# **Label Encoding**

Converting the categories into numerical values.

In [9]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['Category'] = label_encoder.fit_transform(df['Category'])
num_categories = len(label_encoder.classes_)
print(f"Number of unique categories: {num_categories}")

Number of unique categories: 43


# **Data Splitting**

In [10]:
from sklearn.model_selection import train_test_split


x_train, x_test, y_train, y_test = train_test_split(
    df['Text'], df['Category'], test_size=0.2, random_state=42, shuffle=True
)

# **Tokenization**

Using the BertTokenizer for breaking into subword units.
for example

"playing soccer" is broken into
 ['play', '##ing', 'soccer']

 This handles padding and truncation to ensure all input sequences have the same shape, which is required for batch processing in BERT.

In [11]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(x_train.tolist(), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(x_test.tolist(), truncation=True, padding=True, max_length=128)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

# **Wrapping Data**

In [12]:
import torch

class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [13]:
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification

# Wrap tokenized data
train_dataset = TextDataset(train_encodings, y_train.tolist())
val_dataset = TextDataset(val_encodings, y_test.tolist())


In [14]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

You can adjust the batch size accordingly to what resources you have. I monitored the gpu ram usage while training this model increasing the batch size from 16 to 32 to 128

In [20]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments
import torch
import shutil
from google.colab import files


# Check if GPU is available and use it
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Load model and send to GPU
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=num_categories  # Number of classes
).to(device)

# Enhanced training arguments with GPU optimization
training_args = TrainingArguments(
    output_dir='./results',
    report_to="none",  # This disables W&B and other logging integrations
    num_train_epochs=5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    warmup_steps=500,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)



trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

# Train the model
print("Starting training...")
trainer.train()

# Save the final model
model_save_path = "./new_final_model"
trainer.save_model(model_save_path)
print(f"Model saved to {model_save_path}")


results = trainer.evaluate()
print("Evaluation results:", results)

tokenizer.save_pretrained(model_save_path)
print("Tokenizer saved with the model")

shutil.make_archive('new_final_model', 'zip', 'new_final_model')
files.download('new_final_model.zip')

Using device: cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,3.626022,0.093353,0.051273,0.059324,0.093353
2,3.738100,2.649310,0.505228,0.445818,0.574693,0.505228
3,2.946500,1.542688,0.728155,0.713852,0.754503,0.728155
4,1.701400,1.114946,0.776326,0.773856,0.790475,0.776326
5,1.060500,0.966711,0.781927,0.781791,0.792646,0.781927


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Model saved to ./new_final_model


Evaluation results: {'eval_loss': 0.9667109847068787, 'eval_accuracy': 0.7819268110530246, 'eval_f1': 0.7817906736731295, 'eval_precision': 0.7926457893004405, 'eval_recall': 0.7819268110530246, 'eval_runtime': 20.2355, 'eval_samples_per_second': 132.342, 'eval_steps_per_second': 1.038, 'epoch': 5.0}
Tokenizer saved with the model


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>